In [8]:
import os

import numpy as np
import tensorflow as tf

from tensorflow_vgg import vgg19
from tensorflow_vgg import utils

data_dir = 'dogImages/'
contents = os.listdir(data_dir)
classes = [each for each in contents if os.path.isdir(data_dir + each)]

In [9]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

batch_size = 10
codes_list = []
labels = []
batch = []

codes = None

with tf.Session() as sess:   
    vgg = vgg19.Vgg19()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    with tf.name_scope("content_vgg"):
        vgg.build(input_)

    for each in classes:
        print("Starting {} images".format(each))
        class_path = data_dir + each
        files = os.listdir(class_path)
        for ii, file in enumerate(files, 1):
            # Add images to the current batch
            # utils.load_image crops the input images for us, from the center
            img = utils.load_image(os.path.join(class_path, file))
            batch.append(img.reshape((1, 224, 224, 3)))
            labels.append(each)
            
            # Running the batch through the network to get the codes
            if ii % batch_size == 0 or ii == len(files):
                
                # Image batch to pass to VGG network
                
                images = np.concatenate(batch)
                
                feed_dict = {input_: images}
                codes_batch = sess.run(vgg.relu7, feed_dict=feed_dict)
                
                # Here I'm building an array of the codes
                if codes is None:
                    codes = codes_batch
                else:
                    codes = np.concatenate((codes, codes_batch))
                
                # Reset to start building the next batch
                batch = []
                print('{} images processed'.format(ii))

/home/wangyao/Desktop/Transfer_Learning/tensorflow_vgg/vgg19.npy
npy file loaded
build model started
build model finished: 0s
Starting 056.Dachshund images
10 images processed
20 images processed
30 images processed
40 images processed
50 images processed
60 images processed
70 images processed
80 images processed
82 images processed
Starting 100.Lowchen images
10 images processed
20 images processed
30 images processed
40 images processed
42 images processed
Starting 001.Affenpinscher images
10 images processed
20 images processed
30 images processed
40 images processed
50 images processed
60 images processed
70 images processed
80 images processed
Starting 065.Entlebucher_mountain_dog images
10 images processed
20 images processed
30 images processed
40 images processed
50 images processed
53 images processed
Starting 092.Keeshond images
10 images processed
20 images processed
30 images processed
40 images processed
50 images processed
55 images processed
Starting 064.English_toy_spa

In [10]:
# write codes to file
with open('codes_vgg19', 'w') as f:
    codes.tofile(f)
    
# write labels to file
import csv
with open('labels_vgg19', 'w') as f:
    writer = csv.writer(f, delimiter='\n')
    writer.writerow(labels)

In [11]:
# read codes and labels from file
import csv

with open('labels_vgg19') as f:
    reader = csv.reader(f, delimiter='\n')
    labels = np.array([each for each in reader]).squeeze()
with open('codes_vgg19') as f:
    codes = np.fromfile(f, dtype=np.float32)
    codes = codes.reshape((len(labels), -1))

In [12]:
# One-hot encode labels
from sklearn import preprocessing

lb = preprocessing.LabelBinarizer()
lb.fit(labels)
labels_vecs = lb.transform(labels)

In [19]:
# Split the codes and labels into training, validation, and test sets
from sklearn.model_selection import StratifiedShuffleSplit

ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_idx, val_idx = next(ss.split(codes, labels))

half_val_len = int(len(val_idx)/2)
val_idx, test_idx = val_idx[:half_val_len], val_idx[half_val_len:]

train_x, train_y = codes[train_idx], labels_vecs[train_idx]
val_x, val_y = codes[val_idx], labels_vecs[val_idx]
test_x, test_y = codes[test_idx], labels_vecs[test_idx]

In [20]:
print("Train shapes (x, y):", train_x.shape, train_y.shape)
print("Validation shapes (x, y):", val_x.shape, val_y.shape)
print("Test shapes (x, y):", test_x.shape, test_y.shape)

Train shapes (x, y): (6680, 4096) (6680, 133)
Validation shapes (x, y): (835, 4096) (835, 133)
Test shapes (x, y): (836, 4096) (836, 133)


In [21]:
inputs_ = tf.placeholder(tf.float32, shape=[None, codes.shape[1]])
labels_ = tf.placeholder(tf.int64, shape=[None, labels_vecs.shape[1]])

fc = tf.contrib.layers.fully_connected(inputs_, 256)
    
logits = tf.contrib.layers.fully_connected(fc, labels_vecs.shape[1], activation_fn=None)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels_, logits=logits)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer().minimize(cost)

predicted = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [22]:
def get_batches(x, y, n_batches=10):
    """ Return a generator that yields batches from arrays x and y. """
    batch_size = len(x)//n_batches
    
    for ii in range(0, n_batches*batch_size, batch_size):
        if ii != (n_batches-1)*batch_size:
            X, Y = x[ii: ii+batch_size], y[ii: ii+batch_size] 
        else:
            X, Y = x[ii:], y[ii:]
        yield X, Y

In [23]:
epochs = 10
iteration = 0
saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        for x, y in get_batches(train_x, train_y):
            feed = {inputs_: x,
                    labels_: y}
            loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            print("Epoch: {}/{}".format(e+1, epochs),
                  "Iteration: {}".format(iteration),
                  "Training loss: {:.5f}".format(loss))
            iteration += 1
            
            if iteration % 5 == 0:
                feed = {inputs_: val_x,
                        labels_: val_y}
                val_acc = sess.run(accuracy, feed_dict=feed)
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Validation Acc: {:.4f}".format(val_acc))
    saver.save(sess, "checkpoints/dogs.ckpt")

Epoch: 1/10 Iteration: 0 Training loss: 6.16899
Epoch: 1/10 Iteration: 1 Training loss: 4.21672
Epoch: 1/10 Iteration: 2 Training loss: 3.32184
Epoch: 1/10 Iteration: 3 Training loss: 2.61073
Epoch: 1/10 Iteration: 4 Training loss: 2.12746
Epoch: 0/10 Iteration: 5 Validation Acc: 0.6108
Epoch: 1/10 Iteration: 5 Training loss: 1.75416
Epoch: 1/10 Iteration: 6 Training loss: 1.37686
Epoch: 1/10 Iteration: 7 Training loss: 1.35410
Epoch: 1/10 Iteration: 8 Training loss: 0.97576
Epoch: 1/10 Iteration: 9 Training loss: 0.92735
Epoch: 0/10 Iteration: 10 Validation Acc: 0.7820
Epoch: 2/10 Iteration: 10 Training loss: 0.56131
Epoch: 2/10 Iteration: 11 Training loss: 0.56839
Epoch: 2/10 Iteration: 12 Training loss: 0.52722
Epoch: 2/10 Iteration: 13 Training loss: 0.52087
Epoch: 2/10 Iteration: 14 Training loss: 0.48926
Epoch: 1/10 Iteration: 15 Validation Acc: 0.7988
Epoch: 2/10 Iteration: 15 Training loss: 0.47356
Epoch: 2/10 Iteration: 16 Training loss: 0.42094
Epoch: 2/10 Iteration: 17 Train

In [24]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    feed = {inputs_: test_x,
            labels_: test_y}
    test_acc = sess.run(accuracy, feed_dict=feed)
    print("Test accuracy: {:.4f}".format(test_acc))

Test accuracy: 0.8505


In [25]:
import sklearn
from sklearn import svm
from sklearn import cross_validation, grid_search
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.externals import joblib
 
def train_svm_classifer(codes, labels):
    # save 20% of data for performance evaluation
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(codes, labels, train_size=0.8, test_size=0.1)
    print("Train shapes (x, y):", X_train.shape, y_train.shape)
    print("Test shapes (x, y):", X_test.shape, y_test.shape)
    
    clf = svm.SVC(C=1, kernel='linear', probability=True)
    clf.fit(X_train, y_train)
        
    score = clf.score(X_test,y_test)
    print("Test accuracy:", score)

In [26]:
train_svm_classifer(codes,labels)

Train shapes (x, y): (6680, 4096) (6680,)
Test shapes (x, y): (836, 4096) (836,)
Test accuracy: 0.873205741627
